# Movie Predictor
### This project is dedicated to the legendary action hero Steven Seagal.
![title](steven.jpg)
## What does it do?
### Predict
- Average rating of movies on IMDb
### Using
- Directors
- Writers
- Actors
- Cinematographers
- Composers

### Extract the data

In [3]:
import pandas as pd

cols = ["tconst", "averageRating"]
titleAndRating = pd.read_csv('../title.ratings.tsv', delimiter='\t', usecols=cols)

cols = ["tconst", "titleType", "primaryTitle"]
titleTypeAndName = pd.read_csv('../title.basics.tsv', delimiter='\t', usecols=cols)

cols = ["tconst", "nconst", "category"]
movieRoles = pd.read_csv('../title.principles.tsv', delimiter='\t', usecols=cols)

cols = ["nconst", "primaryName"]
names = pd.read_csv('../name.basics.tsv', delimiter='\t', usecols=cols)

### Clean the data so we only have
- Movies
- TV Movies
- Shorts

### And we only need the
- Directors
- Writers
- Actors
- Cinematographers
- Composers

In [4]:

titleTypeAndName = titleTypeAndName.loc[(titleTypeAndName["titleType"] == "short") | (titleTypeAndName["titleType"] == "tvMovie") | (titleTypeAndName["titleType"] == "movie") ]
titleTypeAndName.reset_index(inplace=True,drop=True)
movieRoles = movieRoles.loc[(movieRoles["category"] == "self") | (movieRoles["category"] == "actor") | (movieRoles["category"] == "actress") | (movieRoles["category"] == "director") | (movieRoles["category"] == "writer") | (movieRoles["category"] == "cinematographer") | (movieRoles["category"] == "composer")]
movieRoles.reset_index(inplace=True,drop=True) #Need to reset the index

### Assign the names for convinience

In [5]:
cols = ["nconst"]
movieRoles = movieRoles.join(names.set_index(cols), on = cols)
movieRoles

,tconst,nconst,category,primaryName
0,tt0000001,nm1588970,self,Carmencita
1,tt0000001,nm0005690,director,William K.L. Dickson
2,tt0000001,nm0374658,cinematographer,William Heise
3,tt0000002,nm0721526,director,Émile Reynaud
4,tt0000002,nm1335271,composer,Gaston Paulin
...,...,...,...,...
42219420,tt9916880,nm0286175,actor,Wayne Forester
42219421,tt9916880,nm10535738,actress,Eden Gamliel
42219422,tt9916880,nm0996406,director,Hilary Audus
42219423,tt9916880,nm1482639,writer,Lucinda Whiteley


### Separate the roles

In [6]:
actors = movieRoles[  (movieRoles['category'] == "actor") | (movieRoles['category'] == "actress") | (movieRoles['category'] == "self")]
actors.reset_index(inplace=True,drop=True)
actors.drop(["nconst", "category"], axis=1, inplace=True)
actors.rename(columns = {"primaryName":"actors"}, inplace=True)

composers = movieRoles[  (movieRoles['category'] == "composer")]
composers.reset_index(inplace=True,drop=True)
composers.drop(["nconst", "category"], axis = 1, inplace=True)
composers.rename(columns = {"primaryName":"composers"}, inplace=True)

directors = movieRoles[  (movieRoles['category'] == "director")]
directors.reset_index(inplace=True,drop=True)
directors.drop(["nconst", "category"], axis=1, inplace=True)
directors.rename(columns = {"primaryName":"directors"}, inplace=True)

cinematographers = movieRoles[  (movieRoles['category'] == "cinematographer")]
cinematographers.reset_index(inplace=True,drop=True)
cinematographers.drop(["nconst", "category"], axis=1, inplace=True)
cinematographers.rename(columns = {"primaryName":"cinematographers"}, inplace=True)

writers = movieRoles[  (movieRoles['category'] == "writer")]
writers.reset_index(inplace=True,drop=True)
writers.drop(["nconst", "category"], axis=1, inplace=True)
writers.rename(columns = {"primaryName":"writers"}, inplace=True)


C:\Users\filip\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\filip\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


### Make the new dataset
- titleType, primaryTitle, Directors, Writers, Actors, Cinematographers, Composers, rating

In [7]:
movieData = pd.DataFrame(columns = ['tconst'])
#movieData = pd.DataFrame(columns = ['tconst','titleType', 'primaryTitle', 'director', 'writer', 'actor', 'cinematographer', 'composer', 'averageRating'])

movieData['tconst'] = titleTypeAndName['tconst'].copy()

cols = ["tconst"]
movieData = movieData.join(titleTypeAndName.set_index(cols), on = cols)
movieData = movieData.join(titleAndRating.set_index(cols), on = cols)
movieData = movieData.dropna( how='all', subset=['averageRating'])
movieData.reset_index(inplace=True,drop=True)


### Let's add some professions

In [18]:
movieDataTemp = movieData.join(directors.set_index(cols), on = cols)
#movieDataTemp = movieDataTemp.merge(writers.set_index(cols), left_on ='directors', right_on = 'writers')
movieDataTemp = movieDataTemp.join(actors.set_index(cols), on = cols)
movieDataTemp = movieDataTemp.join(composers.set_index(cols), on = cols)
movieDataTemp = movieDataTemp.join(cinematographers.set_index(cols), on = cols)
movieDataTemp

movieDataTemp.to_csv("movieData.tsv", sep="\t")

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False